# Multi-class SVM

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from mlxtend.plotting.decision_regions import plot_decision_regions

%matplotlib widget

In [2]:
def linear_kernel(x1, x2):
    return x1.T @ x2

def poly_kernel(x1, x2, d = 2, c = 0):
    return (x1 @ x2.T + c)**d


class multiclass_svm():
    def __init__(self, kernel='linear', c=1.0, tol=1e-3, maxiter=1000):
        self._kernel = kernel
        self._tol = tol
        self._maxiter = maxiter
        self.eps = 0.001
        
        if self._kernel == 'linear':
            self._k = linear_kernel
        elif self._kernel == 'poly':
            self._k = poly_kernel
        
        self._c = c
        
    def _init_params(self):
        self._error_cache = np.zeros(self._data.shape[0])
        self._alphas = np.ones(self._data.shape[0]) * .1
        self._b = 0
        
        if self._kernel == 'linear':
            self._weights = np.random.rand(self._data.shape[1])

    def predict_score(self, x):
        """Predicts a raw score (not classification)
        
        Arguments
            x, array (batch_size, n_features) - input samples.
        """
        u = 0
        if self._kernel == 'linear':
            u = self._weights @ x.T - self._b
        else:
            for i in range(self._data.shape[0]):
                u += self._targets[i] * self._alphas[i] * self._k(self._data[i], x)
            u -= self._b
        return u
        
    def predict(self, x):
        """Classifies input samples.
        
        Arguments
            x, array (batch_size, n_features) - input samples.
        """
        score = self.predict_score(x)

        if type(score) is np.ndarray:
            score[score < 0] = -1
            score[score >= 0] = 1

            return score
        else:
            return -1 if score < 0 else 1

    def smo_step(self, i1, i2):
        if i1 == i2:
            return 0

        x1 = self._data[i1]
        x2 = self._data[i2]
        y1 = self._targets[i1]
        y2 = self._targets[i2]
        alpha1 = self._alphas[i1]
        alpha2 = self._alphas[i2]

        # Compute errors for x1 and x2
        e1 = self.predict_score(x1) - y1
        e2 = self.predict_score(x2) - y2

        s = y1 * y2

        if s == 1:
            L = max(0, alpha2 + alpha1 - self._c)
            H = min(self._c, alpha2 + alpha1)
        else:
            L = max(0, alpha2 - alpha1)
            H = min(self._c, self._c + alpha2 - alpha1)

        if L == H:
            return 0

        k11 = self._k(x1, x1)
        k22 = self._k(x2, x2)
        k12 = self._k(x1, x2)

        eta = k11 + k22 - 2 * k12

        if eta > 0:
            a2 = alpha2 + y2 * (e1 - e2) / eta
            if a2 <= L:
                a2 = L
            elif a2 >= H:
                a2 = H
        # TODO: the negative case
        else:
            print(f"[DEBUG] smo_step: eta = {eta}")
            
            f1 = (y1*(e1 + self._b)) - (alpha1 * k11) - (s*alpha2*k12)
            f2 = (y2*(e2 + self._b)) - (s*alpha1*k12) - (alpha2*k22)
            L1 = alpha1 + s*(alpha2 - L)
            H1 = alpha1 + s*(alpha2 - H)
            
            Lobj = (L1*f1) + (L*f2) + (0.5*(L1**2)*k11) + (0.5*(L**2)*k22) + (s*L*L1*k12)
            Hobj = (H1*f1) + (H*f2) + (0.5*(H1**2)*k11) + (0.5*(H**2)*k22) + (s*H*H1*k12)
            
            if (Lobj < Hobj- self.eps):
                a2 = L
            elif (Lobj > (Hobj + self.eps)):
                  a2 = H
            else:
                  a2 = alpha2
            
        if np.abs(a2 - alpha2) < 1e-3 * (a2 + alpha2 + 1e-3):
            return 0

        a1 = alpha1 + s * (alpha2 - a2)

        # Update threshold to reflect change in Lagrange multipliers
        b1 = e1 + y1 * (a1 - alpha1) * k11 + y2 * (a2 - alpha2) * k12 + self._b
        b2 = e2 + y1 * (a1 - alpha1) * k12 + y2 * (a2 - alpha2) * k22 + self._b
        self._b = (b1 + b2) / 2

        # Update weight vector to reflect change in a1 & a2, if SVM is linear
        if self._kernel == 'linear':
            self._weights = np.sum((self._targets * self._alphas)[:, None] * self._data, axis=0)
        
        # Store a1 and a2 in alpha array
        self._alphas[i1] = a1
        self._alphas[i2] = a2

        # update error cache using new multipliers
        for i in range (self._data.shape[0]):
            self._error_cache[i] = self.predict_score(self._data[i]) - self._targets[i]

        return 1

    def examine(self, i2):
        x2 = self._data[i2]
        y2 = self._targets[i2]
        alpha2 = self._alphas[i2]
        e2 = self.predict_score(x2) - y2
        r2 = e2 * y2

        # Heuristic for picking the first multiplier
        if (r2 < -self._tol and alpha2 < self._c) or (r2 > self._tol and alpha2 > 0):
            f_idxs = np.where((self._alphas != 0) & (self._alphas != self._c))[0]

            if len(f_idxs) > 1:
                # Hueristic for second multiplier: get i1 with lowest absolute error |e1 - e2|

                # TODO: Clean this up
                if e2 > 0:
                    min_error = 999999
                    for i, v in enumerate(f_idxs):
                        if v == i2:
                            continue

                        if self._error_cache[v] == 0:
                            self._error_cache[v] = self.predict_score(self._data[v]) - self._targets[v]
                        error = np.abs(e2 - self._error_cache[v])

                        if error < min_error:
                            min_error = error
                            i1 = v
                else:
                    max_error = -999999
                    for i, v in enumerate(f_idxs):
                        if v == i2:
                            continue

                        if self._error_cache[v] == 0:
                            self._error_cache[v] = self.predict_score(self._data[v]) - self._targets[v]
                        error = np.abs(e2 - self._error_cache[v])

                        if error > max_error:
                            max_error = error
                            i1 = v

                if self.smo_step(i1, i2):
                    return 1
                
                # Loop over all non-zero and non-C alpha, starting at random point
                for i, v in enumerate(np.random.permutation(f_idxs)):
                    if self.smo_step(v, i2):
                        return 1
                
                # Loop over all possible i1, starting at a random point
                for i in range(self._data.shape[0]):
                    if i == i2:
                        continue
                    if self.smo_step(i, i2):
                        return 1
                
        return 0
    
    def fit(self, data, targets):
        self._data = data
        self._targets = targets
        
        self._init_params()
        
        n_changed = 0
        examine_all = True
        n_iter = 0
        
        while (n_changed > 0 or examine_all is True) and n_iter < self._maxiter:
            n_changed = 0
            n_iter += 1
            
            if examine_all is True:
                # loop over all training examples
                for i in range(data.shape[0]):
                    n_changed += self.examine(i)
            else:
                # loop over examples where alpha is not 0 & not C
                f_idxs = np.where((self._alphas != 0) & (self._alphas != self._c))[0]
                for i, v in enumerate(f_idxs):
                    n_changed += self.examine(v)
            
            if examine_all is True:
                examine_all = False
            elif n_changed == 0:
                examine_all = True
    
    def acc(self, y_pred,y):
        y_pred = [1 if pred == -1. else 0 for pred in y_pred]
        
        accuracy = np.sum(y == y_pred)/len(y)
        
        return accuracy

In [3]:
data = datasets.load_iris()

X = data.data
y = data.target

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.1)

# Multi - class

## Model 1: Type 0 vs (Type 1 and Type 2)

In [5]:
# Redefine y for model 1
y_for_model1 = []

for i in range(len(y)):
    if(y[i] != 0):
        y_for_model1.append(1)
    else:
        y_for_model1.append(0)
y_for_model1 = np.array(y_for_model1)
print(y_for_model1)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1]


In [6]:
# Split the data into training and testing sets
X_train_m1, X_test_m1, y_train_m1, y_test_m1 = train_test_split(X, y_for_model1, random_state = 42, test_size=0.1)

## Linear Kernel 

In [7]:
#Model 1 creation my model
l_svm_m1 = multiclass_svm(c =5.0)

#Model 1 training and testing
l_svm_m1.fit(X_train_m1, y_train_m1)
print(l_svm_m1._weights)
print(l_svm_m1._b)


pred_m1_svm = l_svm_m1.predict(X_test_m1)
print("Accuracy of linear svm model 1= ", (l_svm_m1.acc(pred_m1_svm, y_test_m1))*100, "%")

[DEBUG] smo_step: eta = 0.0
[41.37329759 18.74931904 28.31130831  8.37465952]
331.87737160435535
Accuracy of linear svm model 1=  13.333333333333334 %


In [8]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC


l_sk_model1 = LinearSVC()

l_sk_model1.fit(X_train_m1, y_train_m1.astype(np.int32))

print(f"coef_={l_sk_model1.coef_}")
print(f"intercept={l_sk_model1.intercept_}")
pred_m1_svc = l_sk_model1.predict(X_test_m1)
print("Accuracy of linear svc model 1= ", accuracy_score(y_test_m1, pred_m1_svc)*100)

coef_=[[-0.18423713 -0.45122995  0.80794366  0.45071881]]
intercept=[-0.10956519]
Accuracy of linear svc model 1=  100.0


## Poly Kernel

In [9]:
p_svm_m1 = multiclass_svm(kernel = 'poly')

#Model 1 training and testing
p_svm_m1.fit(X_train_m1, y_train_m1)
print(p_svm_m1._b)


pred_m1_svm = p_svm_m1.predict(X_test_m1)
print("Accuracy of poly svm model 1= ", (p_svm_m1.acc(pred_m1_svm, y_test_m1))*100, "%")

13904.215100000001
Accuracy of poly svm model 1=  0.0 %


In [10]:
from sklearn.svm import SVC
p_svc_m1 = SVC(kernel = 'poly', degree = 2)
p_svc_m1.fit(X_train_m1, y_train_m1.astype(np.int32))

#print(f"coef_={p_svc.coef_}")
print(f"intercept={p_svc_m1.intercept_}")

pred_m1_svc = p_svc_m1.predict(X_test_m1)
print("Accuracy of poly svc model 1 = ", accuracy_score(y_test_m1, pred_m1_svc)*100)

intercept=[-1.25416081]
Accuracy of poly svc model 1 =  100.0


## Model 2: Type 1 vs (Type 0 and Type 2)

In [11]:
# Redefine y for model 2
y_for_model2 = []

for i in range(len(y)):
    if(y[i] != 1):
        y_for_model2.append(1)
    else:
        y_for_model2.append(0)
y_for_model2 = np.array(y_for_model2)
print(y_for_model2)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1]


In [12]:
# Split the data into training and testing sets
X_train_m2, X_test_m2, y_train_m2, y_test_m2 = train_test_split(X, y_for_model2, random_state = 42, test_size=0.1)

## Linear Kernel

In [13]:
#Model 2 creation my model
l_svm_m2 = multiclass_svm(c =5.0)

#Model 2 training and testing
l_svm_m2.fit(X_train_m2, y_train_m2)
print(l_svm_m2._weights)
print(l_svm_m2._b)


pred_m2_svm = l_svm_m2.predict(X_test_m2)
print("Accuracy of linear svm model 2= ", (l_svm_m2.acc(pred_m2_svm, y_test_m2))*100, "%")

[DEBUG] smo_step: eta = 0.0
[DEBUG] smo_step: eta = 0.0
[DEBUG] smo_step: eta = 0.0
[DEBUG] smo_step: eta = 0.0
[DEBUG] smo_step: eta = 0.0
[1286.16  824.06  534.26  138.24]
13158.036500000002
Accuracy of linear svm model 2=  46.666666666666664 %


In [14]:
l_sk_model2 = LinearSVC()

l_sk_model2.fit(X_train_m2, y_train_m2.astype(np.int32))

print(f"coef_={l_sk_model2.coef_}")
print(f"intercept={l_sk_model2.intercept_}")
pred_m2_svc = l_sk_model2.predict(X_test_m2)
print("Accuracy of linear svc model 1= ", accuracy_score(y_test_m2, pred_m2_svc)*100)

coef_=[[-0.08289923  0.86696848 -0.38737716  0.95124133]]
intercept=[-1.48721473]
Accuracy of linear svc model 1=  73.33333333333333


C:\Users\aadit\anaconda3\envs\cse6363\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


## Poly Kernel

In [15]:
p_svm_m2 = multiclass_svm(kernel = 'poly')

#Model 2 training and testing
p_svm_m2.fit(X_train_m2, y_train_m2)
print(p_svm_m2._b)


pred_m2_svm = p_svm_m2.predict(X_test_m2)
print("Accuracy of poly svm model 2= ", (p_svm_m2.acc(pred_m2_svm, y_test_m2))*100, "%")

[DEBUG] smo_step: eta = 0.0
138580.82085000013
Accuracy of poly svm model 2=  46.666666666666664 %


In [16]:
p_svc_m2 = SVC(kernel = 'poly', degree = 2)
p_svc_m2.fit(X_train_m2, y_train_m2.astype(np.int32))

#print(f"coef_={p_svc.coef_}")
print(f"intercept={p_svc_m2.intercept_}")

pred_m2_svc = p_svc_m2.predict(X_test_m2)
print("Accuracy of poly svc model 2 = ", accuracy_score(y_test_m2, pred_m2_svc)*100)


intercept=[-0.6704262]
Accuracy of poly svc model 2 =  93.33333333333333


## Model 3: Type 2 vs (Type 0 and Type 1)

In [17]:
# Redefine y for model 3
y_for_model3 = []

for i in range(len(y)):
    if(y[i] != 2):
        y_for_model3.append(1)
    else:
        y_for_model3.append(0)
y_for_model3 = np.array(y_for_model3)
print(y_for_model3)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]


In [18]:
# Split the data into training and testing sets
X_train_m3, X_test_m3, y_train_m3, y_test_m3 = train_test_split(X, y_for_model3, random_state = 42, test_size=0.1)

## Linear Kernel

In [19]:
#Model 3 creation my model
l_svm_m3 = multiclass_svm(c =5.0)

#Model 3 training and testing
l_svm_m3.fit(X_train_m3, y_train_m3)
print(l_svm_m3._weights)
print(l_svm_m3._b)


pred_m3_svm = l_svm_m3.predict(X_test_m3)
print("Accuracy of linear svm model 3= ", (l_svm_m3.acc(pred_m3_svm, y_test_m3))*100, "%")

[2395.36 1364.68 1255.55  345.35]
28946.086000000003
Accuracy of linear svm model 3=  86.66666666666667 %


In [20]:
l_sk_model3 = LinearSVC()

l_sk_model3.fit(X_train_m3, y_train_m3.astype(np.int32))

print(f"coef_={l_sk_model3.coef_}")
print(f"intercept={l_sk_model3.intercept_}")
pred_m3_svc = l_sk_model3.predict(X_test_m3)
print("Accuracy of linear svc model 1= ", accuracy_score(y_test_m3, pred_m3_svc)*100)

coef_=[[ 0.7414038   1.14194142 -1.36903246 -1.6526887 ]]
intercept=[1.53419638]
Accuracy of linear svc model 1=  100.0


C:\Users\aadit\anaconda3\envs\cse6363\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


## Poly Kernel

In [21]:
p_svm_m3 = multiclass_svm(kernel = 'poly')

#Model 3 training and testing
p_svm_m3.fit(X_train_m3, y_train_m3)
print(p_svm_m3._b)


pred_m3_svm = p_svm_m3.predict(X_test_m3)
print("Accuracy of poly svm model 3= ", (p_svm_m3.acc(pred_m3_svm, y_test_m3))*100, "%")

431269.76340000005
Accuracy of poly svm model 3=  86.66666666666667 %


In [22]:
p_svc_m3 = SVC(kernel = 'poly', degree = 2)
p_svc_m3.fit(X_train_m3, y_train_m3.astype(np.int32))

#print(f"coef_={p_svc.coef_}")
print(f"intercept={p_svc_m3.intercept_}")

pred_m3_svc = p_svc_m3.predict(X_test_m3)
print("Accuracy of poly svc model 3 = ", accuracy_score(y_test_m3, pred_m3_svc)*100)

intercept=[3.21525098]
Accuracy of poly svc model 3 =  100.0
